In [2]:
# Installation de sklearn
!pip3 install scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 35.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 41.3 MB/s eta 0:00:00a 0:00:01


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import os

def charger_dataset():
    """
    Charge le dataset météo depuis le dossier spécifié
    """
    try:
        chemin = 'WEEK_2/DAY_3/DATASET/weatherHistory.csv'
        if not os.path.exists(chemin):
            print(f"Erreur : Le fichier {chemin} n'existe pas")
            return None
            
        df = pd.read_csv(chemin)
        return df
        
    except Exception as e:
        print(f"Erreur lors du chargement : {e}")
        return None

def explorer_donnees(df):
    """
    Explore le dataset et affiche les informations importantes
    """
    print("\n=== EXPLORATION DES DONNÉES ===")
    print("\nAperçu des premières lignes :")
    print(df.head())
    
    print("\nInformations sur le dataset :")
    print(df.info())
    
    print("\nStatistiques descriptives :")
    print(df.describe())

def normaliser_precipitations(df):
    """
    Effectue la normalisation Min-Max sur les précipitations
    """
    print("\n=== NORMALISATION DES DONNÉES ===")
    scaler = MinMaxScaler()
    
    # Vérification et normalisation de la colonne de précipitations
    if 'Precip' in df.columns:
        df['Precipitation_normalized'] = scaler.fit_transform(df[['Precip']])
        print("\nAperçu des données normalisées :")
        print(df[['Precip', 'Precipitation_normalized']].head())
    else:
        print("La colonne de précipitations n'a pas été trouvée dans le dataset")
    
    return df

def reduction_donnees_pca(df):
    """
    Applique PCA pour réduire la dimensionnalité
    """
    print("\n=== RÉDUCTION DES DONNÉES (PCA) ===")
    
    # Sélection uniquement des colonnes numériques pour PCA
    colonnes_numeriques = df.select_dtypes(include=[np.number]).columns
    print(f"\nColonnes numériques utilisées pour PCA : {colonnes_numeriques.tolist()}")
    
    # Application de PCA
    pca = PCA(n_components=2)
    donnees_reduites = pca.fit_transform(df[colonnes_numeriques])
    
    # Création d'un nouveau DataFrame avec les composantes principales
    df_pca = pd.DataFrame(
        donnees_reduites, 
        columns=['PC1', 'PC2']
    )
    
    print("\nAperçu des données réduites :")
    print(df_pca.head())
    
    # Affichage de la variance expliquée
    print("\nVariance expliquée par composante :")
    print(pca.explained_variance_ratio_)
    
    return df_pca

def agreger_donnees(df):
    """
    Agrège les données par date
    """
    print("\n=== AGRÉGATION DES DONNÉES ===")
    
    try:
        # Vérification du format de la colonne de date
        print("\nVérification du format de date...")
        print("Colonnes disponibles :", df.columns.tolist())
        
        if 'Formatted Date' in df.columns:
            # Affichage du type de données de la colonne
            print("\nType de données de 'Formatted Date':", df['Formatted Date'].dtype)
            print("Exemple de date:", df['Formatted Date'].iloc[0])
            
            # Conversion en datetime avec gestion d'erreurs
            try:
                df['Date'] = pd.to_datetime(df['Formatted Date'])
                
                # Agrégation par date
                df_agrege = df.groupby(df['Date'].date).agg({
                    'Precip': ['mean', 'min', 'max', 'count']
                }).round(3)
                
                print("\nDonnées agrégées par date :")
                print(df_agrege.head())
                
            except Exception as e:
                print(f"Erreur lors de la conversion de la date : {e}")
                # Alternative : agrégation sans conversion de date
                df_agrege = df.groupby('Formatted Date').agg({
                    'Precip': ['mean', 'min', 'max', 'count']
                }).round(3)
                print("\nDonnées agrégées par date (sans conversion) :")
                print(df_agrege.head())
                
        else:
            print("La colonne 'Formatted Date' n'a pas été trouvée dans le dataset")
            print("Colonnes disponibles :", df.columns.tolist())
    
    except Exception as e:
        print(f"Erreur lors de l'agrégation : {e}")
        return None
    
    return df_agrege

def main():
    # Chargement des données
    print("Chargement du dataset météo...")
    df = charger_dataset()
    
    if df is not None:
        # Exploration
        explorer_donnees(df)
        
        # Normalisation
        df = normaliser_precipitations(df)
        
        # Réduction
        df_pca = reduction_donnees_pca(df)
        
        # Agrégation
        df_agrege = agreger_donnees(df)
        
        print("\nAnalyse complétée avec succès !")

if __name__ == "__main__":
    main()


Chargement du dataset météo...

=== EXPLORATION DES DONNÉES ===

Aperçu des premières lignes :
                  Formatted Date        Summary Precip Type  Temperature (C)  \
0  2006-04-01 00:00:00.000 +0200  Partly Cloudy        rain         9.472222   
1  2006-04-01 01:00:00.000 +0200  Partly Cloudy        rain         9.355556   
2  2006-04-01 02:00:00.000 +0200  Mostly Cloudy        rain         9.377778   
3  2006-04-01 03:00:00.000 +0200  Partly Cloudy        rain         8.288889   
4  2006-04-01 04:00:00.000 +0200  Mostly Cloudy        rain         8.755556   

   Apparent Temperature (C)  Humidity  Wind Speed (km/h)  \
0                  7.388889      0.89            14.1197   
1                  7.227778      0.86            14.2646   
2                  9.377778      0.89             3.9284   
3                  5.944444      0.83            14.1036   
4                  6.977778      0.83            11.0446   

   Wind Bearing (degrees)  Visibility (km)  Loud Cover  Pressur

/var/folders/3h/ppgwfhhj1rz7_5v8s2zf1pwr0000gn/T/ipykernel_31457/708517675.py:102: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date'] = pd.to_datetime(df['Formatted Date'])
